In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os
from time import time
from time import sleep
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import threading
from datetime import datetime

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'API-KEY',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)


In [3]:
df = (
    pd.json_normalize(data['data']) 
      .drop(columns=[
          'slug', 'num_market_pairs', 'tags', 'infinite_supply',
          'platform', 'self_reported_circulating_supply',
          'self_reported_market_cap', 'tvl_ratio', 'quote.USD.tvl',
          'platform.symbol', 'platform.slug', 'platform.id',
          'platform.name', 'platform.token_address', 'max_supply',
          'quote.USD.last_updated', 'id', 'quote.USD.volume_change_24h',
          'last_updated'
      ])                          
      .rename(columns={            
          'cmc_rank': 'Rank',
          'quote.USD.price': 'USD_Price',
          'quote.USD.market_cap': 'market_cap',
          'quote.USD.market_cap_dominance': 'market_cap_dominance',
          'quote.USD.fully_dituted_market_cap': 'diluted_market_cap',
          'quote.USD.percent_change_1h': '1h_change',
          'quote.USD.percent_change_24h': '24h_change',
          'quote.USD.percent_change_7d': '7d_change',
          'quote.USD.percent_change_30d': '30d_change',
          'quote.USD.percent_change_60d': '60d_change',
          'quote.USD.percent_change_90d': '90_d_change'
      })
)

df['timestamp'] = pd.to_datetime('now')  # create a new timestamp column :contentReference[oaicite:3]{index=3}

df

,name,symbol,date_added,circulating_supply,total_supply,Rank,USD_Price,quote.USD.volume_24h,1h_change,24h_change,7d_change,30d_change,60d_change,90_d_change,market_cap,market_cap_dominance,quote.USD.fully_diluted_market_cap,timestamp
0,Bitcoin,BTC,2010-07-13T00:00:00.000Z,19859950.000,19859950.000,1,95476.330,15365722632.875,0.007,-0.484,1.664,15.968,5.733,0.398,1896155146059.210,63.871,2005002936424.480,2025-05-04 17:38:46.761344
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,120732619.997,120732619.997,2,1829.823,9091361216.702,0.038,0.214,1.342,2.889,-17.751,-29.594,220919374020.462,7.441,220919374020.460,2025-05-04 17:38:46.761344
2,Tether USDt,USDT,2015-02-25T00:00:00.000Z,149354155091.778,151512481348.111,3,1.000,37747725571.717,-0.003,-0.021,-0.023,0.037,0.027,-0.040,149366459440.726,5.034,151524963507.980,2025-05-04 17:38:46.761344
3,XRP,XRP,2013-08-04T00:00:00.000Z,58503545101.000,99986181777.000,4,2.190,1523683130.969,0.237,-0.020,0.378,7.339,-11.721,-8.384,128124901073.208,4.318,219003653286.330,2025-05-04 17:38:46.761344
4,BNB,BNB,2017-07-25T00:00:00.000Z,140889854.080,140889854.080,5,593.820,1235854507.819,-0.128,-0.497,-1.067,1.063,-0.800,2.725,83663279848.546,2.818,83663279848.550,2025-05-04 17:38:46.761344
5,Solana,SOL,2020-04-10T00:00:00.000Z,518070127.771,599912684.717,6,145.710,1604174329.516,-0.002,-1.135,-1.464,27.088,-1.024,-26.524,75488254600.903,2.543,87413574060.000,2025-05-04 17:38:46.761344
6,USDC,USDC,2018-10-08T00:00:00.000Z,61505450019.748,61505450019.748,7,1.000,5202874805.374,-0.001,-0.014,-0.003,-0.002,-0.008,0.023,61502755075.176,2.072,61502755075.180,2025-05-04 17:38:46.761344
7,Dogecoin,DOGE,2013-12-15T00:00:00.000Z,149122076383.705,149122076383.705,8,0.174,545520945.078,0.055,-2.068,-3.494,7.758,-14.134,-31.886,26013062035.026,0.876,26013062035.030,2025-05-04 17:38:46.761344
8,Cardano,ADA,2017-10-01T00:00:00.000Z,35302796703.744,44995001398.802,9,0.691,945487664.843,-0.093,-1.073,-0.821,9.072,-29.922,-3.274,24383210523.076,0.821,31080950405.890,2025-05-04 17:38:46.761344
9,TRON,TRX,2017-09-13T00:00:00.000Z,94925628957.355,94925632214.274,10,0.248,413412886.781,-0.126,-0.266,-0.909,3.703,1.773,11.050,23501480886.657,0.792,23501481693.000,2025-05-04 17:38:46.761344


In [4]:
def API_Runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
    }
    headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '41adbe98-59d2-4d5a-98df-b331ff828a26',
    }

    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    df2 = (pd.json_normalize(data['data']).drop(columns=[
          'slug', 'num_market_pairs', 'tags', 'infinite_supply',
          'platform', 'self_reported_circulating_supply',
          'self_reported_market_cap', 'tvl_ratio', 'quote.USD.tvl',
          'platform.symbol', 'platform.slug', 'platform.id',
          'platform.name', 'platform.token_address', 'max_supply',
          'quote.USD.last_updated', 'id', 'quote.USD.volume_change_24h',
          'last_updated'
      ]).rename(columns={            
          'cmc_rank': 'Rank',
          'quote.USD.price': 'USD_Price',
          'quote.USD.market_cap': 'market_cap',
          'quote.USD.market_cap_dominance': 'market_cap_dominance',
          'quote.USD.fully_dituted_market_cap': 'diluted_market_cap',
          'quote.USD.percent_change_1h': '1h_change',
          'quote.USD.percent_change_24h': '24h_change',
          'quote.USD.percent_change_7d': '7d_change',
          'quote.USD.percent_change_30d': '30d_change',
          'quote.USD.percent_change_60d': '60d_change',
          'quote.USD.percent_change_90d': '90_d_change'
      }))   
    df2['timestamp'] = pd.to_datetime('now')
    df = pd.concat([df, df2], ignore_index=True)
    if not os.path.isfile(r'E:\my_project\API automation\API.csv'):
        df2.to_csv(r'E:\my_project\API automation\API.csv', header = True)
    else:
        df2.to_csv(r'E:\my_project\API automation\API.csv', mode = 'a', header = False)

In [ ]:
for i in range(333):
    API_Runner()
    print('API Runner Completed')
    sleep(3600)
    break
exit()

API Runner Completed


In [ ]:
df33 = df.groupby('name', sort = False)[['quote.USD.percent_change_1h',
                                        'quote.USD.percent_change_24h',
                                        'quote.USD.percent_change_7d',
                                        'quote.USD.percent_change_30d',
                                        'quote.USD.percent_change_60d',
                                        'quote.USD.percent_change_90d']].mean().rename(columns={'quote.USD.percent_change_1h': '1h',
                                                                                                'quote.USD.percent_change_24h': '24h',
                                                                                                'quote.USD.percent_change_7d': '7d',
                                                                                                'quote.USD.percent_change_30d': '30d',
                                                                                                'quote.USD.percent_change_60d': '60d',
                                                                                                'quote.USD.percent_change_90d': '90d'})
df4 = df33.stack()
df5 = df4.to_frame(name='values')
index = pd.Index(range(900))
df6 = df5.reset_index()
df7 = df6.rename(columns={'level_1': 'Percentage_change'})
df7

In [ ]:
sns.catplot(x='Percentage_change', y='values', hue = 'name', data = df7, kind = 'point', marker = True)

In [ ]:
# Filter df7 to only Bitcoin
df_btc = df7[df7['name'] == 'BNB']

# Then plot
sns.catplot(
    x='Percentage_change',
    y='values',
    hue='name',
    data=df_btc,
    kind='point',
    markers=True,
    ci=None,
    
)
plt.tight_layout()
plt.show()